In [1]:
test_input1 = """....#
#..#.
#..##
..#..
#...."""

print(test_input1)

....#
#..#.
#..##
..#..
#....


In [2]:
rows = test_input1.split('\n')

rows

['....#', '#..#.', '#..##', '..#..', '#....']

In [3]:
def make_grid(rows):
    grid = {}
    for r, row in enumerate(rows):
        for c, char in enumerate(row):
            grid[(r, c)] = {'.': 0, '#': 1}[char]
    return grid

grid = make_grid(rows)

In [4]:
from collections import defaultdict


def make_neighbors(grid):
    neighbors = defaultdict(list)

    for (r, c) in grid:
        for dr, dc in [(-1, 0), (1, 0), (0, 1), (0, -1)]:
            if (r + dr, c + dc) in grid:
                neighbors[(r, c)].append((r + dr, c + dc))
                
    return neighbors

neighbors = make_neighbors(grid)

In [5]:
def get_neighbors(pos, neighbors, grid):
    return sum(grid[n] for n in neighbors[pos])

get_neighbors((0, 3), neighbors, grid)

2

In [6]:
def step(grid, neighbors):
    new_grid = {}
    for pos, bugs in grid.items():
        s = get_neighbors(pos, neighbors, grid)
        if bugs == 1:
            if s != 1:
                new_grid[pos] = 0
            else:
                new_grid[pos] = 1
        else:
            if s >= 1 and s <= 2:
                new_grid[pos] = 1
            else:
                new_grid[pos] = 0
    return new_grid

In [7]:
def print_grid(grid):
    for r in range(5):
        for c in range(5):
            print({0: '.', 1: '#'}[grid[(r, c)]], sep='', end='')
        print()

In [8]:
test_input2 = """#..#.
####.
###.#
##.##
.##.."""

print(test_input1)
print()
print(test_input2)

....#
#..#.
#..##
..#..
#....

#..#.
####.
###.#
##.##
.##..


In [9]:
print_grid(step(grid, neighbors))

#..#.
####.
###.#
##.##
.##..


Let's define a hash function.

In [10]:
def hash_grid(grid):
    return hash(tuple(grid.values()))

hash_grid(grid)

-7440750469560754571

In [11]:
hashes = set()
new_grid = grid.copy()
while True:
    h = hash_grid(new_grid)
    if h in hashes:
        break
    else:
        hashes.add(h)
    new_grid = step(new_grid, neighbors)

In [12]:
h

-3065632452959700355

In [13]:
print_grid(new_grid)

.....
.....
.....
#....
.#...


In [14]:
def biodiversity(grid):
    b = 0
    for (r, c), val in grid.items():
        if val == 1:
            b += 2**(5*r + c)
    return b

biodiversity(new_grid)

2129920

# Let's solve part1 

In [15]:
inp = """#####
.#.#.
.#..#
....#
..###"""

grid = make_grid(inp.split('\n'))
neighbors = make_neighbors(grid)
hashes = set()
new_grid = grid.copy()
while True:
    h = hash_grid(new_grid)
    if h in hashes:
        break
    else:
        hashes.add(h)
    new_grid = step(new_grid, neighbors)

print(f"solution for part1: {biodiversity(new_grid)}")

solution for part1: 30442557


# Part 2 

Let's build recursive grids of a fixed size and then let's propagate all these bugs.

In [16]:
def make_recursive_neighbors(recursion_levels):
    # init existing recursive_grid
    recursive_grid = dict()
    for recursion_level in recursion_levels:
        for r in range(5):
            for c in range(5):
                if (r, c) != (2, 2):
                    recursive_grid[(r, c, recursion_level)] = 0



    recursive_neighbors = defaultdict(set)

    # build "normal" neighbors
    for (r, c, recursion_level) in recursive_grid:
        if (r, c) != (2, 2):
            for dr, dc in [(-1, 0), (1, 0), (0, 1), (0, -1)]:
                if (r + dr, c + dc, recursion_level) in recursive_grid: 
                    recursive_neighbors[(r, c, recursion_level)].add((r + dr, c + dc, recursion_level))

    # build "recursed" neighbors
    for recursion_level in recursion_levels:
        # example case 8
        (r, c) = (1, 2)
        if (0, 0, recursion_level - 1) in recursive_grid:
            for cc in range(5):
                recursive_neighbors[(r, c, recursion_level)].add((0, cc, recursion_level - 1))
                recursive_neighbors[(0, cc, recursion_level - 1)].add((r, c, recursion_level))
        # example case 12
        (r, c) = (2, 1)
        if (0, 0, recursion_level - 1) in recursive_grid:
            for rr in range(5):
                recursive_neighbors[(r, c, recursion_level)].add((rr, 0, recursion_level - 1))
                recursive_neighbors[(rr, 0, recursion_level - 1)].add((r, c, recursion_level))
        # example case 14
        (r, c) = (2, 3)
        if (0, 4, recursion_level - 1) in recursive_grid:
              for rr in range(5):
                recursive_neighbors[(r, c, recursion_level)].add((rr, 4, recursion_level - 1))  
                recursive_neighbors[(rr, 4, recursion_level - 1)].add((r, c, recursion_level))  
        # example case 18
        (r, c) = (3, 2)
        if (4, 0, recursion_level - 1) in recursive_grid:
            for cc in range(5):
                recursive_neighbors[(r, c, recursion_level)].add((4, cc, recursion_level - 1))
                recursive_neighbors[(4, cc, recursion_level - 1)].add((r, c, recursion_level))

    return recursive_grid, recursive_neighbors
        
# let's run the unit test
recursion_levels = list(range(-5, 6)) 
recursive_grid, recursive_neighbors = make_recursive_neighbors(recursion_levels)
grid = make_grid(test_input1.split('\n'))
for (r, c), val in grid.items():
    if (r, c) != (2, 2):
        recursive_grid[(r, c, 0)] = val

new_grid = recursive_grid.copy()
for _ in range(10):
    new_grid = step(new_grid, recursive_neighbors)

sum(items for items in new_grid.values())

99

# Solving for part 2 

In [17]:
recursion_levels = list(range(-200, 200)) 
recursive_grid, recursive_neighbors = make_recursive_neighbors(recursion_levels)
grid = make_grid(inp.split('\n'))
for (r, c), val in grid.items():
    if (r, c) != (2, 2):
        recursive_grid[(r, c, 0)] = val

new_grid = recursive_grid.copy()
for _ in range(200):
    new_grid = step(new_grid, recursive_neighbors)

sum(items for items in new_grid.values())

1987